# IDAES-PSE and Reinforcement Learning (RL): HDA Example Notebook

The purpose of this notebook is to guide the user through the flow sheet design process. For this example, we will be designing a **hydrodealkylation (HDA)** process system, in which toluene reacts with hydrogen gas at high temperatures to form benzene. 

**C<sub>6</sub>H<sub>5</sub>CH<sub>3</sub> + H<sub>2</sub> &rarr; C<sub>6</sub>H<sub>6</sub> + CH<sub>4</sub>** 

In [1]:
from RL_CORE import RL_call
import widgets 

/Users/ericaeggleton/miniconda3/envs/idaes/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/ericaeggleton/miniconda3/envs/idaes/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/ericaeggleton/miniconda3/envs/idaes/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/ericaeggleton/m

Instructions for updating:
non-resource variables are not supported in the long term


/Users/ericaeggleton/miniconda3/envs/idaes/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/ericaeggleton/miniconda3/envs/idaes/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/ericaeggleton/miniconda3/envs/idaes/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/

    idaes.models.unit_models  (deprecated in 2.0.0.alpha0) (called from
    <frozen importlib._bootstrap>:219)
    has been moved to idaes.models.unit_models.pressure_changer  (deprecated
    in 2.0.0.alpha0) (called from <frozen importlib._bootstrap>:219)
    been moved to idaes.models.unit_models.heat_exchanger  (deprecated in
    2.0.0.alpha0) (called from <frozen importlib._bootstrap>:219)
    'idaes.core.solvers.get_solver'.  Please update your import. (deprecated
    in 2.0.0.alpha0) (called from <frozen importlib._bootstrap>:219)


## Select Units

In [2]:
w = widgets.select_units()

Use the dropdowns to select number of units.

In [3]:
display(w)

interactive(children=(Dropdown(description='Heater', index=1, options=('0', '1', '2'), style=DescriptionStyle(…

Check that the resulting list matches your input.

In [4]:
w.result

['inlet_feed',
 'outlet_product',
 'outlet_exhaust',
 'heater_1',
 'heatex_1',
 'StReactor_1',
 'mixer2to1_1',
 'mixer2to1_2',
 'flash_1',
 'splitter1to2_1',
 'compressor_1',
 'expander_1']

In [ ]:
list_inlet_all = []
#for item in list_unit_all:

In [ ]:
class user_inputs:
    def __init__(self):
        self.list_unit_all = build_unit_lists(unit_nums)
        self.list_inlet_all = ['outlet_product.inlet', 'outlet_exhaust.inlet',\
                'mixer2to1_1.inlet_1', 'mixer2to1_1.inlet_2', 'heater_1.inlet',\
                'StReactor_1.inlet', 'flash_1.inlet', 'splitter1to2_1.inlet', \
                'compressor_1.inlet']
        self.list_outlet_all = ['inlet_feed.outlet', 'mixer2to1_1.outlet',\
                'heater_1.outlet', 'StReactor_1.outlet', 'flash_1.liq_outlet',\
                    'flash_1.vap_outlet', 'splitter1to2_1.outlet_1',\
                        'splitter1to2_1.outlet_2','compressor_1.outlet']

## RL Options

Select desired reinforcement learning parameters

First, how many episodes or training steps? (Default: 300,000)
Next, define the starting *e-greedy* value, or whether the model prioritizes **exploration** vs. **highest reward**. A value of 1 means the model only explores, whereas a value of 0 means the model only picks the highest reward. (Default: 0.8)
Then, select the *e-greedy increment* and learning rate and GNN option.

In [ ]:
episodes_widget = widgets.IntSlider(
    value=300000,
    min=0,
    max=500000,
    step=50000,
    description='Episodes:',
    style = {'description_width': 'initial'},
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

greedy_widget = widgets.FloatSlider(
    value=0.8,
    min=0,
    max=1,
    step=0.1,
    description='Greedy:',
    style = {'description_width': 'initial'},
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

greedy_inc_widget = widgets.FloatSlider(
    value=5e-6,
    min=0,
    max=5e-3,
    step=5e-7,
    description='e-greedy increment:',
    style = {'description_width': 'initial'},
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format= '.6f',
)

learning_widget = widgets.FloatSlider(
    value=0.01,
    min=0,
    max=0.1,
    step=0.01,
    description='Learning rate:',
    style = {'description_width': 'initial'},
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format= '.2f',
)

GNN_widget = widgets.Dropdown(
    options=['True', 'False'],
    value='False',
    description= 'GNN Enabled',
    disabled=False,
    style = {'description_width': 'initial'}
)

display(episodes_widget, greedy_widget, greedy_inc_widget, learning_widget, GNN_widget)

In [ ]:
episodes = episodes_widget.value
greedy = greedy_widget.value
greedy_inc = greedy_inc_widget.value
learning = learning_widget.value
GNN = eval(GNN_widget.value)

In [ ]:
P = {'model_restore':False,'model_save':False,'Episode_max_mode':'static','Episode_max':episodes,
          'threshold_learn':-1,'model_index':1,'GNN_enable':False,'learning_rate':learning,
          'reward_decay':0.5,'e_greedy':greedy,'replace_target_iter':200,'memory_size':20000,
          'e_greedy_increment_ref':greedy_inc,'e_greedy_increment_type':'variable',
          'batch_size':32,'e_greedy_increment_intervals':[0,0.4,0.6,0.7,0.8,0.9,1.0],
          'e_greedy_increment_ratios':[1,1,1,1,1,1],'Additianl_step':500}

## Run RL

In [ ]:
RL_call(user_inputs(),P)

## Visualizations

In [ ]:
file1 = open("result_n9/IDAES_unique_feasible_1.png", mode='rb')
image1 = file1.read()
file2 = open("result_n9/IDAES_unique_1.png", mode='rb')
image2 = file2.read()
cases = widgets.Image(
    value=image2,
    format='png',
    width=400,
    height=600,
)
feasible = widgets.Image(
    value=image1,
    format='png',
    width=400,
    height=600,
)

sidebyside = widgets.HBox([cases, feasible])
display.display(sidebyside)

* RL outputs
* **top 10 IDAES flow sheets and rerun**
* Dataframe with flow sheet specs/criteria
    * Overall Mass balance
    * Overall Energy balance
    * Net energy input (process heat input)
    * Yields
        * Hydrogen utilization
    * flow sheet differences
* **Plots**
    * Hydrogen utilization
    * **Cost**
    * Show benefits
        * thermal integration
            * heat wasted
            * pre-heating of inputs
   